# Beam bending calculator

1) Leave *lines 1 to 4 as they are*; they ensure that all necessary modules are imported and active.

2) Indentation matters in Python. From line 5 onwards, there should be *exactly 4 spaces* before each line.

3) Every line beginning with # (the numeral sign) is a comment. They are not executed so you can safely ignore or delete them.

You can then play around choosing different loads and application points.
The current code reproduces [this example](https://www.mathalino.com/reviewer/mechanics-and-strength-of-materials/solution-to-problem-448-relationship-between-load-shear).

Now check what happens if you try more interesting expressions for the distributed loads, for example:

```python
beam.distributed_load("5 + x**2", (0, 3))
```
A double asterisk is how you write a power in python (```"x**3"``` means $x^3$)

**Note:** the quotation marks around mathematical expressions are mandatory.

In [ ]:
%matplotlib inline
from crunching import DistributedLoad, PointLoad, graphics_output

with graphics_output() as (beam, x):
    beam.length(9)           # Beam length
    beam.fixed_support(2)    # x-coordinate of the fixed support
    beam.rolling_support(7)  # x-coordinate of the rolling support

    # Add a point load (20kN pointing downward, at x=3m)
    beam.point_load(-20, 3)
    # Distributed load (10kN/m pointing downward, between x=3m and x=9m)
    beam.distributed_load(-10, (3, 9))
    # Distributed load (20kN/m pointing downward, between x=0m and x=2m)
    beam.distributed_load(-20, (0, 2))
    beam.distributed_load("5 + x**2", (0, 3))

Click the link to download [a pdf file with your case results](./output/test01.pdf "Come on and click! You know you want to.").